# NLP Homework 1
submission by Harshitha B R

Task - Bnary Text Classification for classifying sentiment of reviews

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize

import string
! pip install contractions
import contractions

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:


! pip install bs4 # in case you don't have it installed

# Dataset: https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=c375a0bd2eab138c5b301dc9b16d1f1cd926876e77f87ffb5aaf9bd19e96b2d0
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


## Read Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


reading the data from tsv file using read_csv:


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ANLP/amazon_reviews_us_Office_Products_v1_00.tsv.gz', sep='\t', error_bad_lines=False)

<ipython-input-4-6f2b01c50d79>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ANLP/amazon_reviews_us_Office_Products_v1_00.tsv.gz', sep='\t', error_bad_lines=False)
Skipping line 20773: expected 15 fields, saw 22
Skipping line 39834: expected 15 fields, saw 22
Skipping line 52957: expected 15 fields, saw 22
Skipping line 54540: expected 15 fields, saw 22

Skipping line 80276: expected 15 fields, saw 22
Skipping line 96168: expected 15 fields, saw 22
Skipping line 96866: expected 15 fields, saw 22
Skipping line 98175: expected 15 fields, saw 22
Skipping line 112539: expected 15 fields, saw 22
Skipping line 119377: expected 15 fields, saw 22
Skipping line 120065: expected 15 fields, saw 22
Skipping line 124703: expected 15 fields, saw 22

Skipping line 134024: expected 15 fields, saw 22
Skipping line 153938: expected 15 fields, saw 22
S

In [ ]:
df.head()


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,43081963,R18RVCKGH1SSI9,B001BM2MAC,307809868,"Scotch Cushion Wrap 7961, 12 Inches x 100 Feet",Office Products,5,0.0,0.0,N,Y,Five Stars,Great product.,2015-08-31
1,US,10951564,R3L4L6LW1PUOFY,B00DZYEXPQ,75004341,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0.0,1.0,N,Y,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...,2015-08-31
2,US,21143145,R2J8AWXWTDX2TF,B00RTMUHDW,529689027,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0.0,0.0,N,Y,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it.",2015-08-31
3,US,52782374,R1PR37BR7G3M6A,B00D7H8XB6,868449945,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2.0,3.0,N,Y,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...,2015-08-31
4,US,24045652,R3BDDDZMZBZDPU,B001XCWP34,33521401,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0.0,0.0,N,Y,Four Stars,Gorgeous colors and easy to use,2015-08-31








## Keep Reviews and Ratings

Only columns ['review_body', 'star_rating'] are kept into account going forward. Other columns are dropped.



In [ ]:
df = df[['review_body', 'star_rating']]

In [ ]:
df.head(100)

,review_body,star_rating
0,Great product.,5
1,What's to say about this commodity item except...,5
2,"Haven't used yet, but I am sure I will like it.",5
3,Although this was labeled as &#34;new&#34; the...,1
4,Gorgeous colors and easy to use,4
...,...,...
95,"It is ok, but considering the price plus shipp...",3
96,It has nothing to do with the performance of t...,1
97,Exactly as promised. Totally satisfied.,5
98,Excellent storage! Once I found the instructi...,5


# Data Cleaning



Empty rows are dropped using dropna()

In [ ]:
print((len(df)))
df.dropna(subset=['review_body', 'star_rating'], inplace=True)
print('dropping those that are empty')
print((len(df)))

2640254
dropping those that are empty
2640157


'sentiment' will be the target variable and this is got using 'star_rating'.

'review_body' would be the feature to consider while modelling


In [ ]:
df['sentiment'] = df['star_rating'].apply(lambda rating: 1 if int(rating) >= 4 else 0)


In [ ]:
df.head(20)

,review_body,star_rating,sentiment
0,Great product.,5,1
1,What's to say about this commodity item except...,5,1
2,"Haven't used yet, but I am sure I will like it.",5,1
3,Although this was labeled as &#34;new&#34; the...,1,0
4,Gorgeous colors and easy to use,4,1
5,Perfect for planning weekly meals. Removrd the...,5,1
6,Gold plated fusers are the best! It will never...,5,1
7,I have used these highlighters for my bible fo...,5,1
8,Heavy pen that writes very well. I've had it ...,5,1
9,Not sure if they work but sent quickly and fit...,5,1


Cleaning of data is performed in the following steps:


*   Convert text to lower text
*   Remove contractions in the reviews byexpanding them
*   Remove any trailing or leading spaces
*   Remove HTML tags using regex expression query
*   Remove video tags using a regex expression query
*   Removing Emojis and patterns using an emoji pattern, took help from a stackoverflow post : https://stackoverflow.com/a/38498442
*   Removing non alphabetical values from the reviews using a regex expression
*   Removing punctuatons and special characters








In [ ]:
def cleanReviews(text): # function to clean the reviews
    # print(text)
    if pd.isna(text):
        return ""

    text = text.lower()
    text = contractions.fix(text)
    text = text.strip() #to remoe any trailing and leading spaces in the review
    text = re.sub(r'<.*?>', '', text) # Remove HTML tags (if present)
    text = re.sub(r'<video>.*?</video>', '', text)
    text = re.sub(r'[:;][-]?[)D(|\\/\]3]+', '', text)
    emoji_pattern = re.compile(
        r"[\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]",
        flags=re.UNICODE,
    )

    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    ### https://stackoverflow.com/a/38498442
    text = re.sub(r'[^a-zA-Z\s]', '', text) # Remove special characters, punctuation, and symbols
    text = emoji_pattern.sub(r"", text)

    text = text.translate(str.maketrans('', '', string.punctuation)) #to remove punctuation

    return text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 11.6 MB/s eta 0:00:00


In [ ]:
print(df['review_body'].apply(len).mean())
beforeCleaning = df['review_body'].apply(len).mean()

285.2624162123692


In [ ]:

df['review_body'] = df['review_body'].apply(cleanReviews) # applying clenaning to all the reviews where html tags and special char and punctuations are removed

df.head(20)


,review_body,star_rating,sentiment
0,great product,5,1
1,what is to say about this commodity item excep...,5,1
2,have not used yet but i am sure i will like it,5,1
3,although this was labeled as new the one i rec...,1,0
4,gorgeous colors and easy to use,4,1
5,perfect for planning weekly meals removrd the ...,5,1
6,gold plated fusers are the best it will never ...,5,1
7,i have used these highlighters for my bible fo...,5,1
8,heavy pen that writes very well i have had it...,5,1
9,not sure if they work but sent quickly and fit...,5,1


removing any empty rows after cleaning

In [ ]:
df = df[df['review_body'] != '']
print(len(df))
print(df.head())


2639092
                                         review_body star_rating  sentiment
0                                      great product           5          1
1  what is to say about this commodity item excep...           5          1
2     have not used yet but i am sure i will like it           5          1
3  although this was labeled as new the one i rec...           1          0
4                    gorgeous colors and easy to use           4          1


In [ ]:
print(df['review_body'].apply(len).mean())
afterCleaning = df['review_body'].apply(len).mean()


273.5473325674133


average length of data before and after performing cleaning

In [ ]:
print(str(beforeCleaning)+" , " +str(afterCleaning))

285.2624162123692 , 273.5473325674133


In [ ]:
df

,review_body,star_rating,sentiment
0,great product,5,1
1,what is to say about this commodity item excep...,5,1
2,have not used yet but i am sure i will like it,5,1
3,although this was labeled as new the one i rec...,1,0
4,gorgeous colors and easy to use,4,1
...,...,...,...
2640249,i cannot live anymore whithout my palm iii but...,4,1
2640250,although the palm pilot is thin and compact it...,4,1
2640251,this book had a lot of great content without b...,4,1
2640252,i am teaching a course in excel and am using t...,5,1


# Pre-processing

Data preprocessing is carried out on the dataset by prforming stop words and lemmatization


In [ ]:
print(df['review_body'].apply(len).mean())
beforePreProcessing = df['review_body'].apply(len).mean()

273.5473325674133


## remove the stop words

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

df['review_without_stopwords'] = df['review_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## perform lemmatization  

Lemmatization is the process of reducing a word to its base or dictionary form, known as the lemma.

In [ ]:
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    words = nltk.word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized_text = " ".join(lemmatized_words)
    return lemmatized_text

In [ ]:
df['lemmatized_review'] = df['review_without_stopwords'].apply(lemmatize_text)


In [ ]:
df

,review_body,star_rating,sentiment,review_without_stopwords,lemmatized_review
0,great product,5,1,great product,great product
1,what is to say about this commodity item excep...,5,1,say commodity item except come worldhaving nee...,say commodity item except come worldhaving nee...
2,have not used yet but i am sure i will like it,5,1,used yet sure like,used yet sure like
3,although this was labeled as new the one i rec...,1,0,although labeled new one received clearly used...,although labeled new one received clearly used...
4,gorgeous colors and easy to use,4,1,gorgeous colors easy use,gorgeous color easy use
...,...,...,...,...,...
2640249,i cannot live anymore whithout my palm iii but...,4,1,cannot live anymore whithout palm iii also hat...,can not live anymore whithout palm iii also ha...
2640250,although the palm pilot is thin and compact it...,4,1,although palm pilot thin compact may exceed po...,although palm pilot thin compact may exceed po...
2640251,this book had a lot of great content without b...,4,1,book lot great content without boring monotono...,book lot great content without boring monotono...
2640252,i am teaching a course in excel and am using t...,5,1,teaching course excel using stepbystep books g...,teaching course excel using stepbystep book go...


In [ ]:
print(df['lemmatized_review'].apply(len).mean())
afterPreProcessing = df['lemmatized_review'].apply(len).mean()

169.45235406723222


 ## We form two classes and select 50000 reviews randomly from each class.



To make it easier to hande computationally, we use a subset of the dataset from the bogger dataset, by taking 50000 reviews for each of the two classes of sentiment.

In [ ]:
print(str(beforePreProcessing)+" , "+str(afterPreProcessing))

273.5473325674133 , 169.45235406723222


In [ ]:
sentiment_counts = df['sentiment'].value_counts()

# Print the counts
print("Number of positive (sentiment=1) rows:", sentiment_counts.get(1, 0))
print("Number of negative (sentiment=0) rows:", sentiment_counts.get(0, 0))

Number of positive (sentiment=1) rows: 2000130
Number of negative (sentiment=0) rows: 638962


In [ ]:
positive_count = (df['sentiment'] == 1).sum()
negative_count = (df['sentiment'] == 0).sum()
num_samples = min(50000, positive_count, negative_count)

# Sample the rows for each sentiment value
positive_samples = df[df['sentiment'] == 1].sample(n=num_samples, random_state=11)
negative_samples = df[df['sentiment'] == 0].sample(n=num_samples, random_state=11)


# Combine the sampled rows into a single DataFrame
sampled_df = pd.concat([positive_samples, negative_samples])

# Reset the index of the combined DataFrame
sampled_df.reset_index(drop=True, inplace=True)

print(len(positive_samples))
print(len(negative_samples))
print(positive_samples)
print(negative_samples)


50000
50000
                                               review_body star_rating  \
644175   very cool this was for my daughter who had her...           5   
2138702  it is so hard to find a good qualified binder ...           5   
1753775  i love my rhinestone badge holder i get so man...           4   
500377                                             love it           5   
165337   i use these folders to keep student files i lo...           5   
...                                                    ...         ...   
2014213  i reccommend canon products and their supplies...           4   
2187227  small waterproof and handy to use can be used ...         5.0   
1653687  my favorite mechanical pencil i have been usin...           5   
1597700  wonderful pens they write incredibly smoothly ...           5   
922069   i love this very feminine pen in pink and cros...           5   

         sentiment                           review_without_stopwords  \
644175           1  cool d

In [ ]:
sampled_df
sdf = sampled_df
df=sdf
sdf.head()

,review_body,star_rating,sentiment,review_without_stopwords,lemmatized_review
0,very cool this was for my daughter who had her...,5,1,cool daughter share needle staying hospital re...,cool daughter share needle staying hospital re...
1,it is so hard to find a good qualified binder ...,5,1,hard find good qualified binder cabinet need o...,hard find good qualified binder cabinet need o...
2,i love my rhinestone badge holder i get so man...,4,1,love rhinestone badge holder get many compleme...,love rhinestone badge holder get many compleme...
3,love it,5,1,love,love
4,i use these folders to keep student files i lo...,5,1,use folders keep student files love folders us...,use folder keep student file love folder used ...


# TF-IDF and BoW Feature Extraction

In [ ]:
len(df)


100000

In [ ]:
df.head()

,review_body,star_rating,sentiment,review_without_stopwords,lemmatized_review
0,very cool this was for my daughter who had her...,5,1,cool daughter share needle staying hospital re...,cool daughter share needle staying hospital re...
1,it is so hard to find a good qualified binder ...,5,1,hard find good qualified binder cabinet need o...,hard find good qualified binder cabinet need o...
2,i love my rhinestone badge holder i get so man...,4,1,love rhinestone badge holder get many compleme...,love rhinestone badge holder get many compleme...
3,love it,5,1,love,love
4,i use these folders to keep student files i lo...,5,1,use folders keep student files love folders us...,use folder keep student file love folder used ...


In [ ]:

df['original_review_body'] = df['review_body']
df['review_body'] = df['lemmatized_review']

In [ ]:
df.head()

,review_body,star_rating,sentiment,review_without_stopwords,lemmatized_review,original_review_body
0,cool daughter share needle staying hospital re...,5,1,cool daughter share needle staying hospital re...,cool daughter share needle staying hospital re...,very cool this was for my daughter who had her...
1,hard find good qualified binder cabinet need o...,5,1,hard find good qualified binder cabinet need o...,hard find good qualified binder cabinet need o...,it is so hard to find a good qualified binder ...
2,love rhinestone badge holder get many compleme...,4,1,love rhinestone badge holder get many compleme...,love rhinestone badge holder get many compleme...,i love my rhinestone badge holder i get so man...
3,love,5,1,love,love,love it
4,use folder keep student file love folder used ...,5,1,use folders keep student files love folders us...,use folder keep student file love folder used ...,i use these folders to keep student files i lo...


test_train_split is being used to split the dataset into train (80%) and test(20%) data


In [ ]:
X_text = df['review_body']  # Textual feature
y = df['sentiment']  # Assuming 'sentiment' is your target label (0 for negative, 1 for positive)
X_text_train, X_text_test, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42
)

sklearn methods of CountVectorizer() and TfidfVectorizer() are used to generate TF-IDF(term frequency- inverse document frequency) and BoW(Bag of Words) Representations of the Data.

To prevent data leakage, fit_transform on train feature dataset alone transform method on the test feature dataset.
fit_transform is used intially during vecorization to convert text to numerical features and learn the vocabulary.

transorm is run on new data, and the transformation learnt previously would be used to learn parameters during this fitting phase.

In [ ]:
# creating feature extraction of bag of words
count_vectorizer = CountVectorizer()
X_train_bow = count_vectorizer.fit_transform(X_text_train)
X_test_bow = count_vectorizer.transform(X_text_test)


In [ ]:
# creating feature extraction of tfidf
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_text_train)
X_test_tfidf = tfidf_vectorizer.transform(X_text_test)


Model training

# Perceptron Using Both Features

Perceptron model is initiated and grid search is carried out to find best regularization and maximum iterations needed for optimal running of the training.

training is done on both the features extracted

inference is got using the .predict() function and precision, recall and f1 score are calculated using sklearn.metrics

In [ ]:
perceptron = Perceptron()
param_grid = {
    'alpha': [0.1, 0.01, 0.001],  # Regularization strength
    'max_iter': [100, 500, 1000],  # Maximum no of iterations
}

grid_search = GridSearchCV(perceptron, param_grid, cv=5, scoring='f1')

grid_search.fit(X_train_bow, y_train)# Performing hyperparameter tuning on the training data

best_alpha = grid_search.best_params_['alpha'] # Get the best hyperparameters
best_max_iter = grid_search.best_params_['max_iter']

perceptron_classifier_best = Perceptron(alpha=best_alpha, max_iter=best_max_iter)

perceptron_classifier_best.fit(X_train_bow, y_train)

y_pred = perceptron_classifier_best.predict(X_test_bow)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Perceptron BOW")
print(f"F1-score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")


Perceptron BOW
F1-score: 0.79
Precision: 0.81
Recall: 0.78


In [ ]:
perceptron.fit(X_train_tfidf, y_train)
y_pred_perceptron = perceptron.predict(X_test_tfidf)


y_pred = perceptron.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
# Calculate F1-score, precision, and recall
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Perceptron TFIDF")
print(f"F1-score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

Perceptron TFIDF
F1-score: 0.80
Precision: 0.79
Recall: 0.81


# SVM Using Both Features

LinearSVC was instantiated and model was trained on the features extracted from both TFIDF and BoW

In [ ]:

linear_svc_classifier = LinearSVC()

linear_svc_classifier.fit(X_train_bow, y_train)

# Make predictions on the test data
y_pred = linear_svc_classifier.predict(X_test_bow)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
# Calculate F1-score, precision, and recall
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("SVM BoW")
print(f"Accuracy: {accuracy:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

SVM BoW
Accuracy: 0.83
F1-score: 0.83
Precision: 0.82
Recall: 0.85


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:


linear_svc_classifier.fit(X_train_tfidf, y_train)

y_pred = linear_svc_classifier.predict(X_test_tfidf)

f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("SVM TFIDF")
print(f"F1-score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")


SVM TFIDF
F1-score: 0.84
Precision: 0.84
Recall: 0.84


# Logistic Regression Using Both Features

Logistic regression was instantiated and l2 score, max iterations were played around with to find the optimal hyperparameters to use

In [ ]:
logistic_regression = LogisticRegression(penalty='l2', max_iter=1000)
logistic_regression.fit(X_train_bow, y_train)

y_pred = logistic_regression.predict(X_test_bow)

f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Logistic regression BOW")
print(f"Accuracy: {accuracy:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

Logistic regression BOW
Accuracy: 0.83
F1-score: 0.84
Precision: 0.83
Recall: 0.86


In [ ]:
logistic_regression = LogisticRegression(penalty='l2', max_iter=1000)
logistic_regression.fit(X_train_tfidf, y_train)

y_pred = logistic_regression.predict(X_test_tfidf)

f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Logistic regression TFIDF")
print(f"Accuracy: {accuracy:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

Logistic regression TFIDF
Accuracy: 0.83
F1-score: 0.85
Precision: 0.85
Recall: 0.85


# Naive Bayes Using Both Features

Multinomial Naive Bayes was used to implement Naive Bayes model


In [ ]:
multinomial_nb = MultinomialNB()

param_grid = {'alpha': [0.05, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5]} #range of hyperparameters (alpha) to search

grid_search = GridSearchCV(multinomial_nb, param_grid, cv=5, scoring='f1') # do GridSearchCV for hyperparameter tuning
grid_search.fit(X_train_tfidf, y_train)

best_alpha = grid_search.best_params_['alpha'] # best hyperparameter

multinomial_nb_best = MultinomialNB(alpha=best_alpha) # Train the MultinomialNB classifier with the best hyperparameter
multinomial_nb_best.fit(X_train_tfidf, y_train)

# Make predictions on the test review text
y_pred = multinomial_nb_best.predict(X_test_tfidf)

f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Naive BayesTFIDF")
print(f"F1-score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")


Naive BayesTFIDF
F1-score: 0.82
Precision: 0.83
Recall: 0.81


In [ ]:

grid_search.fit(X_train_bow, y_train)

best_alpha = grid_search.best_params_['alpha']
multinomial_nb_best = MultinomialNB(alpha=best_alpha)
multinomial_nb_best.fit(X_train_bow, y_train)

y_pred = multinomial_nb_best.predict(X_test_bow)

f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Naive Bayes BOW")
print(f"F1-score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

Naive Bayes BOW
F1-score: 0.81
Precision: 0.79
Recall: 0.84
